# Word Embeddings : le modèle Word2Vec

## Imports

In [1]:
import sys

from gensim.models.phrases import Phrases, Phraser
from gensim.models import Word2Vec

import nltk
from nltk.tokenize import wordpunct_tokenize
from unidecode import unidecode

## Chargement et traitement des phrases du corpus

### Création d'un objet qui *streame* les lignes d'un fichier pour économiser de la RAM

In [2]:
class MySentences(object):
    """Tokenize and Lemmatize sentences"""
    def __init__(self, filename):
        self.filename = filename

    def __iter__(self):
        for line in open(self.filename, encoding='utf-8', errors="backslashreplace"):
            yield [unidecode(w.lower()) for w in wordpunct_tokenize(line)]

In [3]:
infile = f"../../data/sents.txt"
sentences = MySentences(infile)

### Détection des bigrams

Article intéressant sur le sujet : https://towardsdatascience.com/word2vec-for-phrases-learning-embeddings-for-more-than-one-word-727b6cf723cf

In [4]:
bigram_phrases = Phrases(sentences)

L'object `phrases` peut être vu comme un large dictionnaire d'expressions multi-mots associées à un score, le *PMI-like scoring*. Ce dictionnaire est construit par un apprentissage sur base d'exemples.
Voir les références ci-dessous :
- https://arxiv.org/abs/1310.4546
- https://en.wikipedia.org/wiki/Pointwise_mutual_information

In [5]:
type(bigram_phrases.vocab)

dict

Il contient de nombreuses clés qui sont autant de termes observés dans le corpus

In [6]:
len(bigram_phrases.vocab.keys())

15852820

Prenons une clé au hasard :

In [7]:
key_ = list(bigram_phrases.vocab.keys())[144]
print(key_)

1q


Le dictionnaire indique le score de cette coocurrence :

In [8]:
bigram_phrases.vocab[key_]

488

Lorsque l'instance de `Phrases` a été entraînée, elle peut concaténer les bigrams dans les phrases lorsque c'est pertinent.

### Conversion des `Phrases` en objet `Phraser`

`Phraser` est un alias pour `gensim.models.phrases.FrozenPhrases`, voir ici https://radimrehurek.com/gensim/models/phrases.html.

Le `Phraser` est une version *light* du `Phrases`, plus optimale pour transformer les phrases en concaténant les bigrams.

In [9]:
bigram_phraser = Phraser(phrases_model=bigram_phrases)

Le `Phraser` est un objet qui convertit certains unigrams d'une liste en bigrams lorsqu'ils ont été identifiés comme pertinents.

### Extraction des trigrams

Nous répétons l'opération en envoyant cette fois la liste de bigrams afin d'extraire les trigrams.

In [10]:
trigram_phrases = Phrases(bigram_phraser[sentences])

In [11]:
trigram_phraser = Phraser(phrases_model=trigram_phrases)

### Création d'un corpus d'unigrams, bigrams, trigrams

In [12]:
corpus = list(trigram_phraser[bigram_phraser[sentences]])

In [13]:
print(corpus[:100])

[['mi', 'imnri', 'r', 'i', '<<', 'i', 'i', 'hmu', "'", 'i', '/', 'tx', "-'", 'l', ':', 'marche', 'tenu', 'hors', 'villa', ',', 'la', '9', '.'], ['--', 'u', 'a', 'ete', 'vaain', 'si', 'teicj', '>>', 'm', 'races_indigenes', 'de', 'fr', '.'], ['31', '<)', 'a', '5s', "'", 'k', '131', 'de', '.'], ['rasa', 'iichakdui', "'", 'te', ',', 'do', '(', 'r', '.', '3s0', 'h', '710', '.', 'taureaux', 'iallsenas', ',>', 'ia', 'u', '\\', '--', 'a', '--', ';', '0ii', '.'], ['hollandais', ',', 'dufr', '.'], ['0', '.'], ['--', 'a', '9', '.--', 'la', 'idto', '-', 'vachei', 'laitieres', ':', 'bn', 'vante', '1q', '.'], ['vendues', '3', '\\', 'au', 'prix', 'la', '410', 'a', '*', '<<', 'i', 'h', '\\;', 'genisses', ',', 'kl', '.'], ["'.", '9', '.'], ['i', 'l', '.', '2', 'i', '.', 'id', '.'], ['da', '370', 'i', '6lutr', '.'], ['marche', 'a', '<', 'u', 'porcs', '.'], ['--', 'categorie', 'de', 'lt', 'ilashtya', ':', "'", '237', 'on', 'vente', ';', 'vendus', '1', 'm', '.', 'do', "'", '2', 'i', '.--', 'a', ';:,', 'l'

## Entrainement d'un modèle Word2Vec sur ce corpus

In [14]:
%%time
model = Word2Vec(
    corpus, # On passe le corpus de ngrams que nous venons de créer
    vector_size=32, # Le nombre de dimensions dans lesquelles le contexte des mots devra être réduit, aka. vector_size
    window=5, # La taille du "contexte", ici 5 mots avant et 5 après le mot observé
    min_count=5, # On ignore les mots qui n'apparaissent pas au moins 5 fois dans le corpus (on peut le mettre à plus avec un très gros corpus)
    workers=4, # Permet de paralléliser l'entraînement du modèle en 4 threads
    epochs=5 # Nombre d'itérations du réseau de neurones sur le jeu de données pour ajuster les paramètres avec la descente de gradient, aka. epochs.
)

CPU times: total: 38min 58s
Wall time: 19min 34s


#### Remarque

Vous voyez ici que l'entrainement du modèle est parallélisé (sur 4 workers).

Lors qu'on parallélise l'entrainement du modèle, 4 modèles "séparés" sont entrainés sur environ un quart des phrases.

Ensuite, les résultats sont agrégés pour ne plus faire qu'un seul modèle.

On ne peut prédire quel worker aura quelle phrase, car il y a des aléas lors de la parallélisation (p. ex. un worker qui serait plus lent, etc.).

Du coup, les valeurs peuvent varier légèrement d'un entrainement à l'autre.

Mais, globalement, les résultats restent cohérents.

### Sauver le modèle dans un fichier

In [15]:
outfile = f"../../data/newspapers.model"
model.save(outfile)

## Explorer le modèle

### Charger le modèle en mémoire

In [16]:
model = Word2Vec.load("../../data/newspapers.model")

### Imprimer le vecteur d'un terme

In [17]:
model.wv["austerite"]

array([ 0.09207283,  0.80301017,  0.46958306,  0.5604196 , -1.7737516 ,
       -0.39442918,  0.61867344,  0.35733265, -0.444706  , -0.56987333,
       -0.27363527, -0.6446241 ,  0.8518311 , -1.1580149 ,  1.0189357 ,
       -0.6069897 , -0.44120166,  2.1885011 ,  0.41769096, -0.46694937,
        0.06696355, -1.2362639 ,  0.14674819,  0.27056235,  0.43863618,
        1.1148549 ,  0.9343853 ,  0.29255092,  0.81182426, -1.3171399 ,
       -0.64658564, -0.549801  ], dtype=float32)

### Calculer la similarité entre deux termes

In [18]:
model.wv.similarity("austerite", "rigueur")

0.5749774

### Chercher les mots les plus proches d'un terme donné

In [19]:
model.wv.most_similar("greve", topn=10)

[('greve_generale', 0.9065925478935242),
 ('population_civile', 0.8554121851921082),
 ('rhenanie', 0.8387497067451477),
 ('ruhr', 0.8355034589767456),
 ('region_parisienne', 0.8191252946853638),
 ('campagne_electorale', 0.8118994235992432),
 ('sarre', 0.8106608390808105),
 ('continuation', 0.8068395853042603),
 ('majeure_partie', 0.804804801940918),
 ('catastrophe', 0.80323326587677)]

In [20]:
model.wv.most_similar("politique", topn=10)

[('doctrine', 0.9164836406707764),
 ('solidarite', 0.9123944640159607),
 ('dictature', 0.9122743606567383),
 ('neutralite', 0.9074138402938843),
 ('politique_francaise', 0.9022654891014099),
 ('legalite', 0.8959481716156006),
 ('discipline', 0.8937190771102905),
 ('politique_exterieure', 0.8915813565254211),
 ('cooperation', 0.8906609416007996),
 ('souverainete', 0.8892900347709656)]

In [21]:
model.wv.most_similar("recession", topn=10)

[('conjoncture', 0.935530424118042),
 ('crise_economique', 0.9256728887557983),
 ('probabilite', 0.9220548272132874),
 ('gestation', 0.9159731268882751),
 ('crise_mondiale', 0.9052231907844543),
 ('consultation_electorale', 0.9045829772949219),
 ('politique_allemande', 0.9045485854148865),
 ('production_nationale', 0.9045461416244507),
 ('crise_politique', 0.8990070223808289),
 ('situation_difficile', 0.8985730409622192)]

In [22]:
model.wv.most_similar("rigueur", topn=10)

[('contradiction', 0.9205888509750366),
 ('prudence', 0.9161743521690369),
 ('consideration', 0.904812753200531),
 ('severite', 0.9016741514205933),
 ('justification', 0.900070071220398),
 ('moderation', 0.8952789902687073),
 ('negative', 0.8931050300598145),
 ('repugnance', 0.8893949389457703),
 ('large_part', 0.8874050378799438),
 ('logique', 0.8866111636161804)]

In [23]:
model.wv.most_similar("austerite", topn=10)

[('heredite', 0.9402556419372559),
 ('utilite', 0.9137066006660461),
 ('erudition', 0.9115763306617737),
 ('incoherence', 0.90940922498703),
 ('intransigeance', 0.9074496626853943),
 ('eclosion', 0.9065663814544678),
 ('instabilite', 0.9059146046638489),
 ('apaisement', 0.9028001427650452),
 ('aboutissement', 0.9013491272926331),
 ('ampleur', 0.8984983563423157)]

### Faire des recherches complexes à travers l'espace vectoriel

In [28]:
print(model.wv.most_similar(positive=['politique', 'economique', 'recession'], negative=['austerite']))

[('vie_economique', 0.8899521231651306), ('situation_economique', 0.8831236362457275), ('communaute_europeenne', 0.8802589774131775), ('productivite', 0.8769223690032959), ('situation_generale', 0.8768086433410645), ('vie_sociale', 0.8720418810844421), ('consolidation', 0.866462230682373), ('paix_sociale', 0.8642968535423279), ('securite_collective', 0.8640726208686829), ('stabilisation', 0.8616712093353271)]


In [38]:

print(model.wv.most_similar(positive=['economie','austerite'], negative=['heredite']))



[('incidence', 0.8523808121681213), ('evaluation', 0.8230135440826416), ('utilite', 0.8167029023170471), ('imposition', 0.8112596869468689), ('elevation', 0.8102540373802185), ('integration', 0.8068400621414185), ('expansion_economique', 0.8058851957321167), ('utilisation', 0.8042657375335693), ('expansion', 0.7998732328414917), ('economie_nationale', 0.787531316280365)]
